In [1]:
import sys
!{sys.executable} -m pip install -e ../
!{sys.executable} -m pip install addict

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///tf/pose3D/src
  Attempting uninstall: ShAReD-Net
    Found existing installation: ShAReD-Net 1.0
    Uninstalling ShAReD-Net-1.0:
      Successfully uninstalled ShAReD-Net-1.0
  Running setup.py develop for ShAReD-Net
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%reset -f

In [3]:
import time
import itertools

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def init_devs():
    tf.config.set_soft_device_placement(True)
    
    options = {
                "layout_optimizer": True,
                "constant_folding": True,
                "shape_optimization": True,
                "remapping": True,
                "arithmetic_optimization": True,
                "dependency_optimization": True,
                "loop_optimization": True,
                "function_optimization": True,
                "debug_stripper": True,
                "disable_model_pruning": False,
                "scoped_allocator_optimization": True,
                "pin_to_host_optimization": True,
                "implementation_selector": True,
                "disable_meta_optimizer": False
              }
    #tf.config.optimizer.set_experimental_options(options)

    
    devs = tf.config.get_visible_devices()
    print(devs)

    print(tf.config.threading.get_inter_op_parallelism_threads())
    print(tf.config.threading.get_intra_op_parallelism_threads())
    tf.config.threading.set_inter_op_parallelism_threads(12)
    tf.config.threading.set_intra_op_parallelism_threads(12)
    print(tf.config.threading.get_inter_op_parallelism_threads())
    print(tf.config.threading.get_intra_op_parallelism_threads())

    gpus = tf.config.experimental.list_physical_devices('GPU')
    gpus = gpus[:] 
    if gpus:
        try:
        # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus, 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
    logical_devs = tf.config.list_logical_devices()
    physical_devs = tf.config.experimental.list_physical_devices()

    print("physical_devs",physical_devs)
    print("logical_devs", logical_devs)
    
    print(tf.version.VERSION)
init_devs()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
0
0
12
12
4 Physical GPUs, 4 Logical GPUs
physical_devs [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:2', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:3', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2'

In [5]:
import ShAReD_Net.training.train_distributed as train

import ShAReD_Net.training.slim as training_slim
import ShAReD_Net.model.slim as model_slim
import ShAReD_Net.training.loss.slim as loss_slim



In [6]:
from ShAReD_Net.configure import config

config.dataset.IMG_PATH = "/dataset/jta/images"
config.dataset.ANNO_PATH = "/dataset/jta/new_image_annotations"

config.checkpoint.path = "/tf/pose3D/checkpoints/run6"
config.tensorboard.path = "/tf/pose3D/logdir/run6"


In [7]:
import ShAReD_Net.data.transform.transform as transform

data_split = "train"

def create_dataset(per_replica_batch_size):
    with tf.device("/cpu:0"):
        train_ds = transform.create_dataset(data_split, 4).shuffle(500).prefetch(100)
    return train_ds


In [8]:
def train_loss():
    loss = training_slim.SlimTrainingLoss()
    return loss
    
def train_model():
    
    low_level_extractor = model_slim.LowLevelExtractor(color_channel=13, texture_channel=16, texture_compositions=16, out_channel=32)

    encoder = model_slim.Encoder(dense_blocks_count=3, dense_filter_count=8)
    
    pos_decoder = model_slim.PosDecoder(dense_blocks_count=2, dense_filter_count=8)
    
    pose_decoder = model_slim.PoseDecoder(keypoints=config.model.output.keypoints, z_bins=config.model.z_bins, dense_blocks_count=3, dense_filter_count=8)
    
    model = training_slim.SlimTrainingModel(low_level_extractor, encoder, pos_decoder, pose_decoder)    
    
    return model

def create_checkpoint(step, optimizer, train_model, train_loss):
    nets = {"low_level_extractor":train_model.low_level_extractor,
            "encoder":train_model.encoder,
            "pos_decoder":train_model.pos_decoder,
            "pose_decoder":train_model.pose_decoder,
            "loss_agg":train_loss.loss_agg,
           }
    ckpt = tf.train.Checkpoint(step=step, optimizer=optimizer, **nets)
    manager = tf.train.CheckpointManager(ckpt, config.checkpoint.path, max_to_keep=50)
    return ckpt, manager

In [9]:
import ShAReD_Net.model.modules.slim as slim_modules

roi_size_img = np.asarray(config.model.roi_size) * config.model.img_downsampling + 1
        
roi_extractor = slim_modules.Roi_Extractor(roi_size=roi_size_img)

def calc_img_index(roiindices):
    new_indexes = roiindices * config.model.img_downsampling
    return new_indexes
    

In [10]:
def print_loss(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    
    extra_loss_sum = tf.reduce_sum(extra_loss) / 10
       
    tf.print("On", dev)
    tf.print("detection_loss", detection_loss_sum)
    
    tf.print("estimator_loss_xy", loss_pos_xy_sum)
    tf.print("estimator_loss_z", loss_pos_z_sum)
    
    tf.print("estimator_loss_var_xy", loss_var_xy_sum)
    tf.print("estimator_loss_var_z", loss_var_z_sum)
    
    tf.print("extra_loss_sum", extra_loss_sum)


def simple_summery(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes) = batch
    tf.print("simple_summery")  
    
    loss_per_batch_sum = tf.reduce_sum(loss_per_batch)
    
    extra_loss_sum = tf.reduce_sum(extra_loss) / 10
    
    tf.summary.scalar(f"detection_loss", detection_loss_sum)
    
    tf.summary.scalar(f"loss_pos_xy_sum", loss_pos_xy_sum)
    tf.summary.scalar(f"loss_pos_z_sum", loss_pos_z_sum)
    
    tf.summary.scalar(f"loss_var_xy_sum", loss_var_xy_sum)
    tf.summary.scalar(f"loss_var_z_sum", loss_var_z_sum)
    
    tf.summary.scalar(f"extra_loss", extra_loss_sum)
    
    tf.summary.scalar(f"agg_loss", loss_per_batch_sum)
    
    mean_grad = tf.zeros(())
    for grad in grads:
        mean_grad += tf.reduce_mean(tf.abs(grad))
    mean_grad /= len(grads)
    
    max_grad = tf.zeros(())
    for grad in grads:
        max_grad = tf.math.maximum(tf.reduce_max(tf.abs(grad)),max_grad)
        
    tf.summary.scalar(f"max_grad", max_grad)
    
    tf.summary.scalar(f"mean_grad", mean_grad)
    
    for k in range(config.model.output.keypoints):
        tf.summary.scalar(f"pose x, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,0]-pose_xyz_gt[:,k,0])))
        tf.summary.scalar(f"pose y, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,0]-pose_xyz_gt[:,k,1])))
        tf.summary.scalar(f"pose z, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,0]-pose_xyz_gt[:,k,2])))
    
    
def complex_summery(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes) = batch
    tf.print("complex_summery")
    
    tf.summary.image("image", img, max_outputs=4)
    
    pos_hm_gt_near = pos_hm_gt[...,0,None]
    pos_hm_gt_fare = pos_hm_gt[...,1,None]
    tf.summary.image("pos_hm_gt_near", pos_hm_gt_near, max_outputs=4)
    tf.summary.image("pos_hm_gt_fare", pos_hm_gt_fare, max_outputs=4)
    
    pos_hm_near = pos_hm[...,0,None]
    pos_hm_fare = pos_hm[...,1,None]
    tf.summary.image("pos_hm_near", pos_hm_near, max_outputs=4)
    tf.summary.image("pos_hm_fare", pos_hm_fare, max_outputs=4)
    
    new_indexes = calc_img_index(roi_indexes)
    pose_imges = roi_extractor([img, new_indexes])
    tf.summary.image("pose_imges", pose_imges, max_outputs=4)
    tf.summary.image(f"pose_hm", tf.reduce_sum(pose_prob_map_xy,axis=1), max_outputs=4)
    
    pose_prob_map_xy = tf.unstack(pose_prob_map_xy, axis=1)
    for i, pose_hm in enumerate(pose_prob_map_xy):
        tf.summary.image(f"pose_hm for keypoint {i}", pose_hm, max_outputs=2)
    
    pose_prob_maps_z = tf.unstack(pose_prob_maps_z, axis=1)
    for i, z_slice in enumerate(pose_prob_maps_z):
        tf.summary.image(f"z_slice for keypoint {i}", z_slice[...,None,None], max_outputs=2)


In [11]:
def save_checkpoint(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    def save():
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    tf.py_function(save,[], [])

def finalize(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    def run():
        print("Finalized")
        tf.Graph.finalize(tf.compat.v1.get_default_graph())
    tf.py_function(run,[], [])


def count_params(dev, step, batch, output, loss, extra_loss, ckpt, manager, train_model, grads):
    print(train_model.count_params())



In [12]:
def batching(dataset, batch_size):
    batched_ds = dataset.batch(batch_size)

    def unragg(img, pos_stuff, roi_indexes, pose_stuff):
        rel_pose, pose_indexes = pose_stuff
        pos_heatmap, weights = pos_stuff

        def unragg_roi_indexes(indexes, row_length):
            new_indexes = np.empty([indexes.shape[0], indexes.shape[1]+1], dtype=np.int32)
            new_indexes[:,1:] = indexes
            i = 0
            b = 0
            for length in row_length[1]:
                index = indexes[i:int(i+length)]
                new_indexes[i:int(i+length),0] = b
                i += length
                b += 1
                
            return new_indexes
        
        def unragg_pose_indexes(indexes, row_length):
            new_indexes = np.empty([indexes.shape[0], indexes.shape[1], indexes.shape[-1]+1], dtype=np.int32)
            new_indexes[:,:,1:] = indexes
            i = 0
            b = 0
            for length in row_length[1]:
                index = indexes[i:int(i+length)]
                new_indexes[i:int(i+length),:,0] = b
                i += length
                b += 1
                
            return new_indexes

        roi_indexes_flat = tf.numpy_function(unragg_roi_indexes, [roi_indexes.flat_values, roi_indexes.nested_row_lengths()], Tout=roi_indexes.dtype)
        pose_indexes_flat = tf.numpy_function(unragg_pose_indexes, [pose_indexes.flat_values, pose_indexes.nested_row_lengths()], Tout=pose_indexes.dtype)

        rel_pose_flat =  rel_pose.flat_values

        return img, (pos_heatmap, weights), roi_indexes_flat, (rel_pose_flat, pose_indexes_flat)

    unragged_ds = batched_ds.map(unragg).prefetch(100)
    return unragged_ds


In [13]:
def input_pre(batch):
    img, (pos_heatmap, weights), roi_indexes, (rel_pose, pose_indexes) = batch
    return img, roi_indexes, pose_indexes
    
def loss_pre(output, batch):
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes) = batch
    return (poses_xyz, pose_xyz_gt, pose_prob_map_xy, pose_prob_maps_z), (pos_hm, pos_hm_gt, loss_weights)

def grad_pre(loss, extra_loss, batch, optimizer, train_model, train_loss):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    
    extra_loss_sum = tf.reduce_sum(extra_loss) / 100
    
    loss_per_batch_sum = loss_per_batch
        
    trainable_vars =  train_model.low_level_extractor.trainable_variables + train_model.encoder.trainable_variables + train_model.pos_decoder.trainable_variables + train_model.pose_decoder.trainable_variables+ train_loss.trainable_variables
        
    return loss_per_batch, extra_loss_sum, trainable_vars


#grad_norm = loss_slim.GradNormLayer()

def opt_pre(loss, extra_loss, batch, optimizer, train_model, train_loss):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    
    extra_loss_sum = tf.reduce_sum(extra_loss) / 10
    
    detector = loss_per_batch[0]
    pose = loss_per_batch[1]
    var = train_model.encoder.stage3.trainable_variables
    
    detector_grad = optimizer.get_gradients(detector, var)
    pose_grad = optimizer.get_gradients(pose, var)
    weighted_loss, weighting_loss = grad_norm([loss_per_batch, (detector_grad, pose_grad)])
    
    agg_loss = weighted_loss + weighting_loss
    
    trainable_vars = train_model.low_level_extractor.trainable_variables + train_model.encoder.trainable_variables + train_model.pos_decoder.trainable_variables + train_model.pose_decoder.trainable_variables+ train_loss.trainable_variables
    return agg_loss, extra_loss, trainable_vars

In [14]:
dist_strat = tf.distribute.MirroredStrategy(cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())

steps = 100000

step_callbacks = train.standart_callbacks()
step_callbacks.every_steps[20] = print_loss
step_callbacks.every_steps[200] = save_checkpoint
step_callbacks.every_steps[10] = simple_summery
step_callbacks.every_steps[50] = complex_summery

step_callbacks.at_step[1] = count_params
step_callbacks.at_step[2] = finalize

step_callbacks.make_batches = batching
step_callbacks.grad_pre = grad_pre
step_callbacks.input_pre = input_pre
step_callbacks.loss_pre = loss_pre

step_callbacks.create_dataset = create_dataset
step_callbacks.create_ckpt = create_checkpoint
step_callbacks.create_loss = train_loss
step_callbacks.create_model = train_model


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [15]:
train.train(steps, dist_strat, batch_size = config.training.batch_size, learning_rate=config.training.learning_rate, callbacks = step_callbacks)


Restored from /tf/pose3D/checkpoints/run6/ckpt-20
Instructions for updating:
renamed to `run`
SlimInferenzModel (TensorShape([None, None, None, 3]), TensorShape(None), TensorShape(None))
Extractor (None, None, None, 3)
Encoder (None, None, None, 32)
stage1 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
big_shared1 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
scale_down (None, None, None, 32)
ResAttention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
Attention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
DenseModule (None, None, None, 128)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_up (None, None, None, 48)
big_normal (None, None, None, 48)
normal_shared1 [TensorShape([None, Non

scale_up (None, None, None, 48)
medium_normal (None, None, None, 48)
normal_shared3 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
scale_down (None, None, None, 32)
ResAttention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
Attention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
DenseModule (None, None, None, 128)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_up (None, None, None, 48)
normal_big (None, None, None, 48)
big_shared3 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
scale_down (None, None, None, 32)
ResAttention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
Attention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
Dens

DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
scale_up (None, None, None, 40)
normal_medium (None, None, None, 40)
medium_shared1 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]
scale_down (None, None, None, 64)
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]
Attention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]
DenseModule (None, None, None, 256)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
scale_up (None, None, None, 40)
medium_small (None, None, None, 40)
small_shared1 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]
scale_down (None, None, None, 64)
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]
Attention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 40])]


DenseModule (None, None, None, 448)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_up (None, None, None, 48)
medium_normal (None, None, None, 48)
normal_shared3 [TensorShape([None, 31, 31, 112]), TensorShape([None, None, None, 96])]
scale_down (None, 31, 31, 112)
ResAttention [TensorShape([None, None, None, 112]), TensorShape([None, None, None, 96])]
Attention [TensorShape([None, None, None, 112]), TensorShape([None, None, None, 96])]
DenseModule (None, None, None, 448)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_up (None, None, None, 48)
normal_big (None, None, None, 48)
big_shared3 [TensorShape([None, 

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Expected image (JPEG, PNG, or GIF), got empty file
	 [[{{node DecodePng}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[while/body/_53/while/IteratorGetNextAsOptional_2]]
	 [[while/body/_53/while/StatefulPartitionedCall/StatefulPartitionedCall/Nadam/Nadam/update_1168/update_3/AssignVariableOp_1/_110190]]
  (1) Invalid argument:  Expected image (JPEG, PNG, or GIF), got empty file
	 [[{{node DecodePng}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[while/body/_53/while/IteratorGetNextAsOptional_2]]
0 successful operations.
3 derived errors ignored. [Op:__inference_train_loop_1785124]

Function call stack:
train_loop -> train_loop


estimator_loss_z 228960.422
estimator_loss_xy 15820.8545

detection_loss 98.055069


estimator_loss_var_z 15143.8232
estimator_loss_var_xy 124911

cp 606